**Import Libraries**

In [ ]:
from __future__ import print_function
import math
from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

**Read CSV File and Randomize**

In [ ]:
#How likely to be Republican
votes_dataframe = pd.read_csv("vote_csv.csv")
votes_dataframe = votes_dataframe.replace("y", 1)
votes_dataframe = votes_dataframe.replace("n", 0)
votes_dataframe = votes_dataframe.replace("republican", 1)
votes_dataframe = votes_dataframe.replace("democrat", 0)

#Fill Unfilled Data
#votes_dataframe = votes_dataframe.fillna(0.5)

#Drop Unfilled Data
votes_dataframe.dropna(inplace=True)

votes_dataframe = votes_dataframe.reindex(np.random.permutation(votes_dataframe.index))

**Create Training and Validation Data**

In [ ]:
def preprocess_features(votes_dataframe):
    output_features = votes_dataframe.iloc[:,0:16]
    
    return output_features

In [ ]:
#Single Panda Series
def preprocess_targets(votes_dataframe):
    output_targets = votes_dataframe["Class"]

    return output_targets

In [ ]:
training_split = int(0.6 * len(votes_dataframe))
validations_split = int(0.3 * len(votes_dataframe))

#Panda Data
training_examples = preprocess_features(votes_dataframe.iloc[0:training_split])
training_targets = preprocess_targets(votes_dataframe.iloc[0:training_split])
validation_examples = preprocess_features(votes_dataframe.iloc[training_split:training_split + validations_split])
validation_targets = preprocess_targets(votes_dataframe.iloc[training_split:training_split + validations_split])
#Do not touch until end!
test_examples = preprocess_features(votes_dataframe.iloc[training_split + validations_split:])
test_targets = preprocess_targets(votes_dataframe.iloc[training_split + validations_split:])

**Training Functions**

In [ ]:
#From Google Crash Course
#Create Tensorflow Columns
def construct_feature_columns(input_features):
  """Construct the TensorFlow Feature Columns.

  Args:
    input_features: The names of the numerical input features to use.
  Returns:
    A set of feature columns
  """ 
  return set([tf.feature_column.numeric_column(my_feature)
              for my_feature in input_features])

In [ ]:
#Return next batch (Of type of Tensorflow Dataset)
def input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), targets))
    
    if(shuffle):
        dataset = dataset.shuffle(1000)
    
    dataset = dataset.batch(batch_size).repeat(num_epochs)
    
    return dataset

In [ ]:
def train_linear_classifier_model(
    learning_rate,
    steps,
    batch_size,
    training_examples,
    training_targets,
    validation_examples,
    validation_targets):
    
    periods = 10
    steps_per_period = steps / periods
    
    #Initialize Linear Classifier
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    optimizer = tf.contrib.estimator.clip_gradients_by_norm(optimizer, 5.0)
    linear_classifier = tf.estimator.LinearClassifier(
        feature_columns=construct_feature_columns(training_examples),
        optimizer=optimizer
    )
    
    #Training Functions
    training_input_fn = lambda: input_fn(
        training_examples,
        training_targets,
        batch_size=batch_size) 
    predict_training_input_fn = lambda: input_fn(
        training_examples, 
        training_targets, 
        num_epochs=1, 
        shuffle=False)
    
    #Validation Function
    predict_validation_input_fn = lambda: input_fn(
        validation_examples, 
        validation_targets, 
        num_epochs=1, 
        shuffle=False)
    
    #Train Model
    training_log_losses = []
    validation_log_losses = []
    
    print("Training Model")
    for period in range(0, periods):
        linear_classifier.train(
            input_fn=training_input_fn,
            #Manually break total steps by 10
            steps=steps_per_period
        )
        
        #Use Sklearn to calculate Log Loss
        training_probabilities = linear_classifier.predict(input_fn=predict_training_input_fn)
        training_probabilities = np.array([item['probabilities'][0] for item in training_probabilities])
        training_log_loss = metrics.log_loss(training_targets, training_probabilities)
        
        #Calculate Validation Log Loss
        validation_probabilities = linear_classifier.predict(input_fn=predict_validation_input_fn)
        validation_probabilities = np.array([item['probabilities'][0] for item in validation_probabilities])
        validation_log_loss = metrics.log_loss(validation_targets, validation_probabilities)
        
        #Append Losses
        training_log_losses.append(training_log_loss)
        validation_log_losses.append(validation_log_loss)
        
        print("Period:", period, "Log Loss:", training_log_loss)
    print("Training Finished")
    
    #Graph
    plt.ylabel("Log Loss")
    plt.xlabel("Periods")
    plt.title("Log Loss vs. Periods")
    plt.tight_layout()
    plt.plot(training_log_losses, label="training")
    plt.plot(validation_log_losses, label="validation")
    plt.legend()
    
    return linear_classifier

In [ ]:
def train_dnn_classifier_model(
    learning_rate,
    steps,
    batch_size,
    hidden_units,
    training_examples,
    training_targets,
    validation_examples,
    validation_targets):

    periods = 10
    steps_per_period = steps / periods
    
    #Initialize DNN Regressor
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    optimizer = tf.contrib.estimator.clip_gradients_by_norm(optimizer, 5.0)
    dnn_classifier = tf.estimator.DNNClassifier(
        feature_columns=construct_feature_columns(training_examples),
        n_classes=2,
        hidden_units=hidden_units,
        optimizer=optimizer,
    )
    
    #Training Functions
    training_input_fn = lambda: input_fn(
        training_examples,
        training_targets,
        batch_size=batch_size) 
    predict_training_input_fn = lambda: input_fn(
        training_examples, 
        training_targets, 
        num_epochs=1, 
        shuffle=False)
    
    #Validation Function
    predict_validation_input_fn = lambda: input_fn(
        validation_examples, 
        validation_targets, 
        num_epochs=1, 
        shuffle=False)
    
    #Train Model
    training_log_losses = []
    validation_log_losses = []
    
    print("Training Model")
    for period in range(0, periods):
        dnn_classifier.train(
            input_fn=training_input_fn,
            #Manually break total steps by 10
            steps=steps_per_period
        )
        
        #Use Sklearn to calculate Log Loss
        training_probabilities = dnn_classifier.predict(input_fn=predict_training_input_fn)
        training_probabilities = np.array([item['probabilities'][0] for item in training_probabilities])
        training_log_loss = metrics.log_loss(training_targets, training_probabilities)
        
        #Calculate Validation Log Loss
        validation_probabilities = dnn_classifier.predict(input_fn=predict_validation_input_fn)
        validation_probabilities = np.array([item['probabilities'][0] for item in validation_probabilities])
        validation_log_loss = metrics.log_loss(validation_targets, validation_probabilities)
        
        #Append Losses
        training_log_losses.append(training_log_loss)
        validation_log_losses.append(validation_log_loss)
        
        print("Period:", period, "Log Loss:", training_log_loss)
    print("Training Finished")
    
    #Graph
    plt.ylabel("Log Loss")
    plt.xlabel("Periods")
    plt.title("Log Loss vs. Periods")
    plt.tight_layout()
    plt.plot(training_log_losses, label="training")
    plt.plot(validation_log_losses, label="validation")
    plt.legend()
    
    return dnn_classifier

**Training**

In [ ]:
linear_classifier = train_linear_classifier_model(
    learning_rate=0.0000005,
    steps=750,
    batch_size=20,
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets)

In [ ]:
dnn_classifier = train_dnn_classifier_model(
    learning_rate=0.0000005,
    steps=750,
    batch_size=20,
    hidden_units=[3, 2],
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets)

**Test Data**

In [ ]:
def test_model(model, test_examples, test_targets):
    predict_test_input_fn = lambda: input_fn(
        test_examples, 
        test_targets, 
        num_epochs=1, 
        shuffle=False)
    
    test_predictions = model.predict(input_fn=predict_test_input_fn)
    test_predictions = np.array([item['probabilities'][0] for item in test_predictions])

    for prediction, label in zip(test_predictions, test_targets): 
        if(prediction > 0.5 and label or prediction < 0.5 and not label):
            print("Probabilities:", prediction, "Label:", label)
        else:
            print("Probabilities:", prediction, "Label:", label, "WRONG")
    
    return test_predictions

In [ ]:
test_probabilities_linear_classifier = test_model(linear_classifier, test_examples, test_targets)
test_log_loss_linear_classifier = metrics.log_loss(test_targets, test_probabilities_linear_classifier)
print("Test Log Loss:", test_log_loss_linear_classifier)

In [ ]:
test_probabilities_dnn_classifier = test_model(dnn_classifier, test_examples, test_targets)
test_log_loss_dnn_classifier = metrics.log_loss(test_targets, test_probabilities_dnn_classifier)
print("Test Log Loss:", test_log_loss_dnn_classifier)

**Additional Info**

In [ ]:
#Linear Classifier Weights
for v in linear_classifier.get_variable_names():
    print(v, linear_classifier.get_variable_value(v))